# BigQuery

In [1]:
import pandas as pd

In [2]:
from google.cloud import bigquery

In [3]:
#for colab users
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
# Create a "Client" object
client = bigquery.Client(project="week4-bigscaleanalytics")

In [4]:
# Construct a reference to the week4 dataset
dataset_ref = client.dataset("Week4_datasets", project="week4-bigscaleanalytics")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [5]:
# List all the tables in the  dataset
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset 
for table in tables:  
    print(table.table_id)

Players
Teams


In [6]:
# Construct a reference to the "Teams" table
table_ref = dataset_ref.table("Teams")

# API request - fetch the table
teams_table = client.get_table(table_ref)

table_ref = dataset_ref.table("Players")
players_table = client.get_table(table_ref)

In [7]:
# Print information on all the columns in the "Teams" table 
players_table.schema

[SchemaField('surname', 'STRING', 'NULLABLE', None, ()),
 SchemaField('team', 'STRING', 'NULLABLE', None, ()),
 SchemaField('position', 'STRING', 'NULLABLE', None, ()),
 SchemaField('minutes', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('shots', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('passes', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('tackles', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('saves', 'INTEGER', 'NULLABLE', None, ())]

In [16]:
# Preview the first five lines of the "Teams" table. You can do this with pandas
client.list_rows(teams_table, max_results=5).to_dataframe()

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
0,Honduras,38,3,0,1,2,0,3,7,0
1,North Korea,105,3,0,0,3,1,12,2,0
2,Greece,13,3,1,0,2,2,5,5,0
3,Cameroon,19,3,0,0,3,2,5,5,0
4,New Zealand,78,3,0,3,0,2,2,6,0


In [41]:
# an example query
q1 = """
select * from `week4-bigscaleanalytics.Week4_datasets.Teams`
where ranking < 20
order by ranking asc
"""

query_job = client.query(q1)
query_job.to_dataframe()

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
0,Brazil,1,5,3,1,1,9,4,7,2
1,Spain,2,6,5,0,1,7,2,3,0
2,Portugal,3,4,1,2,1,7,1,8,1
3,Netherlands,4,6,6,0,0,12,5,15,0
4,Italy,5,3,0,2,1,4,5,5,0
5,Germany,6,6,4,0,2,13,3,8,1
6,Argentina,7,5,4,0,1,10,6,7,0
7,England,8,4,1,2,1,3,5,6,0
8,France,9,3,0,1,2,1,4,6,1
9,Greece,13,3,1,0,2,2,5,5,0


In [20]:
# join the two data sets
q2 = """
select teams.*, players.surname, players.position, players.shots,
players.minutes, players.passes, players.tackles, players.saves
from `week4-bigscaleanalytics.Week4_datasets.Teams` teams
inner join `week4-bigscaleanalytics.Week4_datasets.Players` players
on teams.team = players.team
"""

query_job = client.query(q2)
query_job.to_dataframe().head(10)

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards,surname,position,shots,minutes,passes,tackles,saves
0,Honduras,38,3,0,1,2,0,3,7,0,Jerry Palacios,forward,0,84,19,0,0
1,Honduras,38,3,0,1,2,0,3,7,0,Walter Martinez,forward,0,125,43,1,0
2,Honduras,38,3,0,1,2,0,3,7,0,Chavez,defender,0,270,62,3,0
3,Honduras,38,3,0,1,2,0,3,7,0,Izaguirre,defender,0,180,51,8,0
4,Honduras,38,3,0,1,2,0,3,7,0,Mauricio Sabilluen,defender,0,90,38,5,0
5,Honduras,38,3,0,1,2,0,3,7,0,Mendoza,defender,0,180,51,3,0
6,Honduras,38,3,0,1,2,0,3,7,0,Valladares,goalkeeper,0,270,51,0,12
7,Honduras,38,3,0,1,2,0,3,7,0,Espinoza,midfielder,0,135,36,4,0
8,Honduras,38,3,0,1,2,0,3,7,0,Guevara,midfielder,0,156,60,6,0
9,Honduras,38,3,0,1,2,0,3,7,0,Thomas,midfielder,0,114,37,5,0


In [57]:
# practice more queries on your own
# you can even use the examples from week 2

## Exercise:
Let's run some queries on the accidents data of 2016 from the nhtsa traffic facilities dataset. This data-set is publicly available in google cloud.

In [8]:
# Construct a reference to the traffic facilities dataset
traffic_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")

# API request - fetch the dataset
traffic_dataset = client.get_dataset(traffic_ref)

In [9]:
# Construct a reference to the "accident_2016" table
accidents_ref = traffic_ref.table("accident_2016")

# API request - fetch the table
accidents_table = client.get_table(accidents_ref)

In [10]:
accidents_table.schema

[SchemaField('state_number', 'INTEGER', 'NULLABLE', 'This data element identifies the state in which the crash occurred. The codes are from the General Services Administration’s (GSA) publication of worldwide Geographic Location Codes (GLC). For more info on the codes, please look at <C1/V1/D1/PC1/P1/NM1 State Number> section in the pdf: https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/812315', ()),
 SchemaField('state_name', 'STRING', 'NULLABLE', 'This data element identifies the state in which the crash occurred. The codes are from the General Services Administration’s (GSA) publication of worldwide Geographic Location Codes (GLC).', ()),
 SchemaField('consecutive_number', 'INTEGER', 'NULLABLE', 'This data element is the unique case number assigned to each crash. It appears on each data file and is used to merge information from the data files together. xxxxxx Two Characters for State Code followed by Four Characters for Case Number', ()),
 SchemaField('number_of_vehicle_f

In [11]:
# Preview the first five lines of the "accident_2016" table
client.list_rows(accidents_table, max_results=5).to_dataframe()

,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,56,Wyoming,560099,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,0,2016-12-04 03:45:00+00:00
1,16,Idaho,160226,1,1,0,0,0,3,3,...,99,0,None,0,None,0,None,2,0,2016-12-16 16:33:00+00:00
2,5,Arkansas,50006,1,1,0,0,0,2,2,...,99,20,Police-Pursuit-Involved (Since 1994),0,None,0,None,1,0,2016-01-06 03:36:00+00:00
3,48,Texas,482027,1,1,0,0,0,5,5,...,35,0,None,0,None,0,None,1,0,2016-08-07 15:28:00+00:00
4,12,Florida,121570,1,1,0,0,0,5,5,...,99,0,None,0,None,0,None,1,0,2016-07-21 19:37:00+00:00


1- how many crashes happened in each month?

In [14]:
# how many crashes per month
q3 = """
select month_of_crash, count(*) as count
from `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016`
group by month_of_crash
order by month_of_crash asc
"""

query_job_3 = client.query(q3)
query_job_3.to_dataframe()



,month_of_crash,count
0,1,2354
1,2,2426
2,3,2694
3,4,2713
4,5,3005
5,6,3025
6,7,3025
7,8,3134
8,9,3154
9,10,3287


2- Let's call the accidents with more that 5 motor vehichels in transport involved as massive_crash and otherwise call them small crash. Then return the number of these type of crashes for each weather condition.

__Hint: use columns `atmospheric_conditions_1_name` and `number_of_motor_vehicles_in_transport_mvit`.__

In [26]:
q4 = """
select atmospheric_conditions_1_name, (case when number_of_motor_vehicles_in_transport_mvit > 5 THEN 'massive_crash'
ELSE 'small_crash' END) as crash_type, count(*) as num_crashes
from `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016`
group by atmospheric_conditions_1_name, crash_type
order by atmospheric_conditions_1_name
"""

query_job_4 = client.query(q4)
query_job_4.to_dataframe()

,atmospheric_conditions_1_name,crash_type,num_crashes
0,"Blowing Sand, Soil, Dirt",small_crash,20
1,Blowing Snow,small_crash,10
2,Clear,small_crash,24893
3,Clear,massive_crash,63
4,Cloudy,small_crash,5112
5,Cloudy,massive_crash,14
6,"Fog, Smog, Smoke",small_crash,328
7,"Fog, Smog, Smoke",massive_crash,1
8,Freezing Rain or Drizzle,small_crash,17
9,Not Reported,small_crash,1552


3 - Find the top 5 states with the most total number of drunk drivers involved in accidents.

In [27]:
# find the top 5 states with the most number of drunk drivers
q5 = """
select state_name, sum(number_of_drunk_drivers) as sum_drunk_drivers
from `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016`
group by state_name
order by sum_drunk_drivers desc
limit 5
"""

query_job_5 = client.query(q5)
query_job_5.to_dataframe()

,state_name,sum_drunk_drivers
0,California,1052
1,Texas,918
2,Florida,665
3,North Carolina,399
4,Ohio,365


4- Let's see which states, working with a cell-phone is a major cause of accidents. For this you need to use `distract_2016` table. return the state names together with the number of crashes where the driver was distracted "while manipulating cellular phone".

In [61]:
q6 = """
select count(distract.driver_distracted_by) as num_incidents, accident.state_name
from `bigquery-public-data.nhtsa_traffic_fatalities.distract_2016` distract
inner join `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016` accident
on distract.consecutive_number = accident.consecutive_number
where driver_distracted_by = 6
group by state_name
order by num_incidents desc
limit 5
"""

query_job_6 = client.query(q6)
query_job_6.to_dataframe()

,num_incidents,state_name
0,18,Texas
1,9,Missouri
2,9,California
3,8,North Carolina
4,8,Georgia


In [63]:
q6_1 = """
with join_acc_dist as (select distract.driver_distracted_by, accident.state_name
from `bigquery-public-data.nhtsa_traffic_fatalities.distract_2016` distract
inner join `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016` accident
on distract.consecutive_number = accident.consecutive_number)

select count(driver_distracted_by) as num_incidents, state_name from join_acc_dist
where driver_distracted_by = 6
group by state_name
order by num_incidents desc
limit 5
"""

query_job_6_1 = client.query(q6_1)
query_job_6_1.to_dataframe()

,num_incidents,state_name
0,18,Texas
1,9,California
2,9,Missouri
3,8,North Carolina
4,8,Georgia


5- sometimes a "live animal" is a reason that a driver maneuvers and this cause an accidents. Let's see for which months of the year accidents with such a cause happen more. you need to use the `maneuver_2016` table. Return the month and the number of incidents. 

In [54]:
q7 = """
select count(maneuver.driver_maneuvered_to_avoid) as num_incidents, accident.month_of_crash
from `bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016` maneuver
inner join `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016` accident
on maneuver.consecutive_number = maneuver.consecutive_number
where driver_maneuvered_to_avoid = 3
group by month_of_crash
order by num_incidents desc
"""

query_job_7 = client.query(q7)
query_job_7.to_dataframe()

,num_incidents,month_of_crash
0,384579,10
1,369018,9
2,366678,8
3,355797,11
4,353925,7
5,353925,6
6,351585,5
7,338130,12
8,317421,4
9,315198,3


6- In which months of the year in the state of Alaska, "poor road conditions" cause more accidents. Again use the `maneuver_2016` table. return the month and the number of incidents.

In [56]:
q8 = """
select count(maneuver.driver_maneuvered_to_avoid) as num_incidents, accident.month_of_crash
from `bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016` maneuver
inner join `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016` accident
on maneuver.consecutive_number = maneuver.consecutive_number
where driver_maneuvered_to_avoid = 3 and accident.state_name = "Alaska"
group by month_of_crash
order by num_incidents desc
"""

query_job_8 = client.query(q8)
query_job_8.to_dataframe()

,num_incidents,month_of_crash
0,1053,7
1,1053,11
2,936,8
3,936,6
4,819,10
5,819,4
6,702,1
7,702,12
8,702,5
9,702,9


7- For the vehichels having a "brake system" problem which factors where the major reasons that caused the driver to maneuver. Return the `driver_maneuvered_to_avoid_name` column together with the number of incidents for each maneuver reason. You need to use the `factor_2016` table.

In [59]:
q9 = """
select count(maneuver.consecutive_number) as num_incidents, maneuver.driver_maneuvered_to_avoid_name
from `bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016` maneuver
inner join `bigquery-public-data.nhtsa_traffic_fatalities.factor_2016` factor
on maneuver.consecutive_number = factor.consecutive_number
where factor.contributing_circumstances_motor_vehicle = 2
group by driver_maneuvered_to_avoid_name
order by num_incidents desc
"""

query_job_9 = client.query(q9)
query_job_9.to_dataframe()

,num_incidents,driver_maneuvered_to_avoid_name
0,97,Driver Did Not Maneuver To Avoid
1,93,Not Reported
2,25,Motor Vehicle
3,5,Phantom/Non-Contact Motor Vehicle
4,3,No Driver Present/Unknown if Driver Present
5,1,Live Animal


8- Sometimes a non-motorist is ivolved in a crash. Find the number of crashes in which an "inattentive" person was involved in each state. return the state name and number of crashes in the descending order of number of crashes.

In [60]:
q10 = """
select count(nmcrash.consecutive_number) as num_incidents, accident.state_name
from `bigquery-public-data.nhtsa_traffic_fatalities.nmcrash_2016` nmcrash
inner join `bigquery-public-data.nhtsa_traffic_fatalities.accident_2016` accident
on nmcrash.consecutive_number = accident.consecutive_number
where nmcrash.non_motorist_contributing_circumstances = 6
group by state_name
order by num_incidents desc
"""

query_job_10 = client.query(q10)
query_job_10.to_dataframe()

,num_incidents,state_name
0,17,Louisiana
1,16,Georgia
2,14,North Carolina
3,12,Missouri
4,9,Washington
5,8,California
6,7,Ohio
7,7,Texas
8,6,New Jersey
9,4,Florida
